# 範例

實作本篇提到的三大概念

*data augmentation*

除了增加資料量的 Image Augmentation 以外

幾何轉換的概念也經常應用在架構上的設計或是前處理與後處理

* 透過縮放做 upsample / downsample
* 透過 affine transform 做匹配處理
* 透過 perspective transform 做對齊處理

**transformation**

- 翻轉：實作上下翻轉
- 縮放：實作鄰近差值
- 平移：建立 Translation Transformation Matrix 來做平移

In [1]:
import cv2
import time
import numpy as np

img_path = 'lena.png'
img = cv2.imread(img_path)

## 上下翻轉圖片

flip (start, end, step)

In [2]:
# 垂直翻轉 (vertical)
img_vflip = img[::-1, :, :] # shape0 flip image[y,x,channel]

# 組合 + 顯示圖片
hflip = np.vstack((img, img_vflip))
while True:
    cv2.imshow('flip image', hflip)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## 縮放圖片

### 放大

我們先透過縮小圖片去壓縮原有圖片保有的資訊，再放大比較不同方法之間的速度與圖片品質

In [9]:
# 將圖片縮小成原本的 20%
img_test = cv2.resize(img, None, fx=0.2, fy=0.2) #cv2.resize(img,shape,fx,fy,interpolation) shape and fx/fy擇一

'''
其中縮放操作的方式 (interpolation) 會影響處理的速度與圖片品質：

cv2.INTER_NEAREST: 最鄰近插點法
cv2.INTER_LINEAR: 雙線性插補(預設)
cv2.INTER_AREA: 臨域像素再取樣插補
cv2.INTER_CUBIC: 雙立方插補，4×4大小的補點
cv2.INTER_LANCZOS4: Lanczos插補，8×8大小的補點


因為縮小跟放大要參考周圍的 pixel 值
經過統計與運算去減少 / 生成新的 pixel 值

預設縮放皆 INTER_LINEAR
建議縮小用 INTER_AREA
建議放大用 INTER_CUBIC (slow) 或是 INTER_LINEAR
'''


# 將圖片放大為"小圖片"的 8 倍大 = 原圖的 1.6 倍大
fx, fy = 8, 8

# 鄰近差值 scale + 計算花費時間
start_time = time.time()
img_area_scale = cv2.resize(img_test, None, fx=fx, fy=fy, interpolation=cv2.INTER_NEAREST)
print('INTER_NEAREST zoom cost {}'.format(time.time() - start_time))

# 組合 + 顯示圖片
orig_img = cv2.resize(img, img_area_scale.shape[:2])
img_zoom = np.hstack((orig_img, img_area_scale))
while True:
    cv2.imshow('zoom image', img_zoom)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

INTER_NEAREST zoom cost 0.000988006591796875


## 平移幾何轉換

選轉矩陣

In [4]:
# 設定 translation transformation matrix
# x 平移 100 pixel; y 平移 50 pixel
M = np.array([[1, 0, 100],
              [0, 1, 50]], dtype=np.float32)
shift_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

# 組合 + 顯示圖片
img_shift = np.hstack((img, shift_img))
while True:
    cv2.imshow('shift image', img_shift)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break